# Best Practices for Using Runners

In [1]:
%pip uninstall -y todd_ai
%pip install --no-build-isolation --extra-index-url https://pypi.org/simple .. > /dev/null


Found existing installation: todd-ai 0.4.0
Uninstalling todd-ai-0.4.0:
  Successfully uninstalled todd-ai-0.4.0
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pathlib
import tempfile
import time
from pprint import pprint
from typing import Any, NoReturn, TypedDict

import torch
import torch.nn.functional as F
import torch.utils.data
from torch import nn

import todd
from todd.runners import Memo

[2024-02-23 18:26:49,338 35174:140704541179520][patches.py:9 todd <module>] INFO: `ipdb` is installed. Using it for debugging.
/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


## Preparation

### Models

In [3]:
@todd.ModelRegistry.register_()
class RunnerModel(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self._weight = torch.nn.Parameter(torch.tensor(0.0))

    @property
    def weight(self) -> torch.nn.Parameter:
        return self._weight

    def _forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * self._weight

    def forward(
        self,
        runner: todd.runners.BaseRunner,
        batch,
        memo: Memo,
        *args,
        **kwargs,
    ) -> Memo:
        log: dict[str, Any] | None = memo.get("log")
        y = self._forward(batch["x"])
        loss = F.l1_loss(y, batch["y"])
        memo["loss"] = loss
        if log is not None:
            log["batch"] = str(batch)
            log["weight"] = f"{self._weight.item():.3f}"
            log["loss"] = f"{loss:.3f}"
        return memo

### Datasets

In [4]:
class Sample(TypedDict):
    x: int
    y: int

In [5]:
@todd.DatasetRegistry.register_()
class RunnerDataset(torch.utils.data.Dataset[int]):

    def __init__(self, n: int) -> None:
        self._data = list(range(1, n + 1))

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, index: int) -> Sample:
        x = self._data[index]
        return Sample(x=x, y=x * 2)

In [6]:
class Batch(TypedDict):
    x: torch.Tensor
    y: torch.Tensor

## Validators

In [7]:
config = todd.Config(
    type='Validator',
    name='validator',
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree $work_dirs


[2024-02-23 18:26:50,988 35174:140704541179520][base.py:57 todd.Validator.validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpl9zy4w3o
└── validator

2 directories, 0 files


In [8]:
config = todd.Config(
    type='Validator',
    name='validator',
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[dict(type='LogCallback', interval=5)],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree $work_dirs


[2024-02-23 18:26:51,346 35174:140704541179520][base.py:57 todd.Validator.validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:26:51,388 35174:140704541179520][log.py:93 todd.Validator.validator after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-23 18:26:51,392 35174:140704541179520][log.py:93 todd.Validator.validator after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-23 18:26:51,395 35174:140704541179520][log.py:93 todd.Validator.validator after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-23 18:26:51,398 35174:140704541179520][log.py:93 todd.Validator.validator after_run_iter] INFO: Iter [20/20] batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpy_xrnx2v
└── validator

2 directories, 0 files


## Trainers

### Iteration Based

In [9]:
config = todd.Config(
    type="IterBasedTrainer",
    name="iter_based_trainer",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[dict(type="LogCallback", interval=1)],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-23 18:26:51,739 35174:140704541179520][base.py:57 todd.IterBasedTrainer.iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:26:51,743 35174:140704541179520][log.py:93 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [1/8] batch={'x': tensor([1, 3]), 'y': tensor([2, 6])} weight=0.000 loss=4.000
[2024-02-23 18:26:51,744 35174:140704541179520][log.py:93 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [2/8] batch={'x': tensor([5, 8]), 'y': tensor([10, 16])} weight=0.000 loss=13.000
[2024-02-23 18:26:51,745 35174:140704541179520][log.py:93 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [3/8] batch={'x': tensor([4, 2]), 'y': tensor([8, 4])} weight=0.000 loss=6.000
[2024-02-23 18:26:51,748 35174:140704541179520][log.py:93 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [4/8] batch={'x': tensor([ 6, 10]), 'y': tensor([12, 20])} weight=0.000 loss=16.000
[2024-02-2

### Epoch Based

In [10]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="epoch_based_trainer",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[dict(type="LogCallback", interval=1)],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-23 18:26:51,767 35174:140704541179520][base.py:57 todd.EpochBasedTrainer.epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:26:51,768 35174:140704541179520][log.py:99 todd.EpochBasedTrainer.epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2024-02-23 18:26:51,771 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [1/15] batch={'x': tensor([4, 9]), 'y': tensor([ 8, 18])} weight=0.000 loss=13.000
[2024-02-23 18:26:51,772 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [2/15] batch={'x': tensor([7, 3]), 'y': tensor([14,  6])} weight=0.000 loss=10.000
[2024-02-23 18:26:51,774 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [3/15] batch={'x': tensor([10,  2]), 'y': tensor([20,  4])} weight=0.000 loss=12.000
[2024-02-23 18:26:51,775 35174:140704541179520][log.py:93 todd.E

## Callbacks

### Log

In [11]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            collect_env=dict(verbose=False),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-23 18:26:51,926 35174:140704541179520][log.py:55 todd.Validator.log_callback init] INFO: 
platform: macOS-14.0
nvidia_smi: None
python_version: 3.11.7 (main, Dec  4 2023, 18:10:11) [Clang 15.0.0 (clang-1500.1.0.2.5)]
pytorch_version: 2.0.1
torchvision_version: 0.15.2
opencv_version: 4.7.0
todd_version: 0.4.0
cuda_home: None
git_commit_id: 8dccf62
git_status: 
M pyproject.toml
 M todd/runners/base.py
 M todd/runners/epoch_based_trainer.py
 M todd/runners/iter_based_trainer.py
 M todd/runners/trainer.py
 M todd/runners/validator.py
 M todd/utils/torch.py
[2024-02-23 18:26:51,927 35174:140704541179520][base.py:57 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:26:51,931 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-23 18:26:51,934 35174:140704541179520][log.py:93 todd.Validator.log_callback after_ru

In [12]:
config = todd.Config(
    type='Validator',
    name='log_callback',
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type='LogCallback',
            interval=5,
            with_file_handler=True,
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo
    !cat {work_dirs}/log_callback/*.log


[2024-02-23 18:26:51,979 35174:140704541179520][base.py:57 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:26:51,987 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-23 18:26:51,991 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-23 18:26:52,005 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-23 18:26:52,032 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpe_f9kq5u
└── log_callback
    └── 2024-02-23T18-26-51_978897-08-00.log

2 directories, 1 file

[2024-02-23 18:26:51,979 35174:140704541179520][base.py:57 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:26:51,987 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-23 18:26:51,991 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-23 18:26:52,005 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-23 18:26:52,032 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] batch={'x':

In [13]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            eta=dict(type="AverageETA"),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.module.register_forward_hook(
        lambda *args, **kwargs: time.sleep(0.1)
    )
    runner.run()

[2024-02-23 18:26:52,618 35174:140704541179520][base.py:57 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:26:53,137 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:01 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-23 18:26:53,652 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:01 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-23 18:26:54,162 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] ETA 0:00:00 batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-23 18:26:54,678 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] ETA 0:00:00 batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000


In [14]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            eta=dict(type="EMA_ETA", ema=dict(decay=0.2)),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.module.register_forward_hook(
        lambda *args, **kwargs: time.sleep(0.1 * min(10, runner.iter_))
    )
    runner.run()

[2024-02-23 18:26:54,690 35174:140704541179520][base.py:57 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:26:56,210 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:04 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-23 18:27:00,228 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:05 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-02-23 18:27:05,240 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] ETA 0:00:03 batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-02-23 18:27:10,261 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] ETA 0:00:00 batch={'x': tensor([20]), 'y': tensor([40])} weight=0.000 loss=40.000


In [15]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            collect_env=dict(verbose=False),
            with_file_handler=True,
            eta=dict(type="AverageETA"),
        ),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-23 18:27:10,384 35174:140704541179520][log.py:55 todd.Validator.log_callback init] INFO: 
platform: macOS-14.0
nvidia_smi: None
python_version: 3.11.7 (main, Dec  4 2023, 18:10:11) [Clang 15.0.0 (clang-1500.1.0.2.5)]
pytorch_version: 2.0.1
torchvision_version: 0.15.2
opencv_version: 4.7.0
todd_version: 0.4.0
cuda_home: None
git_commit_id: 8dccf62
git_status: 
M pyproject.toml
 M todd/runners/base.py
 M todd/runners/epoch_based_trainer.py
 M todd/runners/iter_based_trainer.py
 M todd/runners/trainer.py
 M todd/runners/validator.py
 M todd/utils/torch.py
[2024-02-23 18:27:10,386 35174:140704541179520][base.py:57 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:27:10,390 35174:140704541179520][log.py:93 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:00 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-02-23 18:27:10,393 35174:140704541179520][log.py:93 todd.Validator.log_callb

### Git

In [16]:
config = todd.Config(
    type="Validator",
    name="git_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(type="GitCallback", diff='HEAD -- ":(exclude)*.ipynb"'),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )

    !echo
    !cat {work_dirs}/git_callback/*.log

[2024-02-23 18:27:10,464 35174:140704541179520][git.py:41 todd.Validator.git_callback init] INFO: Saving git diff to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpk3uvcez8/git_callback/git_diff_2024-02-23T18-27-10_464496-08-00.log
[2024-02-23 18:27:10,467 35174:140704541179520][base.py:57 todd.Validator.git_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R



diff --git a/pyproject.toml b/pyproject.toml
index aa38442..487cb54 100644
--- a/pyproject.toml
+++ b/pyproject.toml
@@ -133,6 +133,7 @@ module = [
     'ipdb.*',
     'mmcv.*',
     'pptx.*',
+    'setuptools.*',
     'torchvision.*',
     'yapf.*',
 ]
diff --git a/todd/runners/base.py b/todd/runners/base.py
index fda811f..e16bbfd 100644
--- a/todd/runners/base.py
+++ b/todd/runners/base.py
@@ -2,6 +2,7 @@ __all__ = [
     'BaseRunner',
 ]
 
+from abc import abstractmethod
 import contextlib
 import getpass
 import logging
@@ -105,8 +106,9 @@ class BaseRunner(StateDictMixin, Generic[T]):
         return self._logger
 
     @property
+    @abstractmethod
     def iters(self) -> int:
-        return len(self._dataloader)
+        pass
 
     def _build_strategy(
         self,
@@ -255,7 +257,7 @@ class BaseRunner(StateDictMixin, Generic[T]):
         return memo
 
     def _setup(self) -> Memo:
-        return dict(dataloader=iter(self._dataloader))
+        return dict()
 
     def _t

### Optimize

In [17]:
config = todd.Config(
    type="IterBasedTrainer",
    name="optimize_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

[2024-02-23 18:27:10,779 35174:140704541179520][base.py:57 todd.IterBasedTrainer.optimize_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:27:10,784 35174:140704541179520][log.py:93 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([5, 8]), 'y': tensor([10, 16])} weight=0.000 loss=13.000
[2024-02-23 18:27:10,786 35174:140704541179520][log.py:93 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([9, 1]), 'y': tensor([18,  2])} weight=0.032 loss=9.838
[2024-02-23 18:27:10,787 35174:140704541179520][log.py:93 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [3/8] batch={'x': tensor([6, 3]), 'y': tensor([12,  6])} weight=0.057 loss=8.741
[2024-02-23 18:27:10,789 35174:140704541179520][log.py:93 todd.IterBasedTrainer.optimize_callback after_run_iter] INFO: Iter [4/8] batch={'x': tensor([10,  2]), 'y': tensor([20,  4])} weight=0.080 loss=11.520
[2024-02-23

### Learning Rate Schedule

In [18]:
config = todd.Config(
    type="IterBasedTrainer",
    name="lr_schedule_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScheduleCallback",
            lr_scheduler=dict(type="LinearLR", total_iters=5),
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-23 18:27:10,810 35174:140704541179520][base.py:57 todd.IterBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:27:10,812 35174:140704541179520][log.py:93 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([6, 3]), 'y': tensor([12,  6])} weight=0.000 loss=9.000 lr=['1.667e-03']
[2024-02-23 18:27:10,814 35174:140704541179520][log.py:93 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([2, 9]), 'y': tensor([ 4, 18])} weight=0.008 loss=10.959 lr=['2.333e-03']
[2024-02-23 18:27:10,815 35174:140704541179520][log.py:93 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [3/8] batch={'x': tensor([7, 5]), 'y': tensor([14, 10])} weight=0.020 loss=11.878 lr=['3.000e-03']
[2024-02-23 18:27:10,817 35174:140704541179520][log.py:93 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [4/8] batch={'x': tensor([8,

In [19]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="lr_schedule_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=4),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScheduleCallback",
            lr_scheduler=dict(type="LinearLR", total_iters=3),
            by_epoch=True,
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=5,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-23 18:27:10,836 35174:140704541179520][base.py:57 todd.EpochBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:27:10,836 35174:140704541179520][log.py:99 todd.EpochBasedTrainer.lr_schedule_callback before_run_epoch] INFO: Epoch [1/5]
[2024-02-23 18:27:10,839 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [1/10] batch={'x': tensor([2, 1]), 'y': tensor([4, 2])} weight=0.000 loss=3.000 lr=['1.667e-03']
[2024-02-23 18:27:10,841 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [2/10] batch={'x': tensor([4, 3]), 'y': tensor([8, 6])} weight=0.002 loss=6.991 lr=['1.667e-03']
[2024-02-23 18:27:10,842 35174:140704541179520][log.py:99 todd.EpochBasedTrainer.lr_schedule_callback before_run_epoch] INFO: Epoch [2/5]
[2024-02-23 18:27:10,844 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.lr_schedule_callback after

### Learning Rate Scale

In [20]:
config = todd.Config(
    type="IterBasedTrainer",
    name="lr_scale_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScaleCallback",
            lr_scaler=dict(base_batch_size=1),
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-02-23 18:27:10,874 35174:140704541179520][lr.py:93 todd.IterBasedTrainer.lr_scale_callback _scale_lr] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2024-02-23 18:27:10,875 35174:140704541179520][base.py:57 todd.IterBasedTrainer.lr_scale_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:27:10,877 35174:140704541179520][log.py:93 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([3, 7]), 'y': tensor([ 6, 14])} weight=0.000 loss=10.000
[2024-02-23 18:27:10,879 35174:140704541179520][log.py:93 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([4, 9]), 'y': tensor([ 8, 18])} weight=0.050 loss=12.675
[2024-02-23 18:27:10,881 35174:140704541179520][log.py:93 todd.IterBasedTrainer.lr_scale_callback after_run_iter] INFO: Iter [3/8] batch={'x': tensor([1, 6]), 'y': tensor([ 2, 12])} weight=0.115 loss=6.598
[2024-02-23 18:27:10,882 35174:140704541179520][log.py:93 

### Checkpoint

In [21]:
config = todd.Config(
    type="IterBasedTrainer",
    name="checkpoint_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_5 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_5'
    for f in iter_5.glob('*.pth'):
        print(f'{f.name}:')
        pprint(torch.load(f, 'cpu'))
        print()

    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_5),
        )
    runner.run()


[2024-02-23 18:27:10,910 35174:140704541179520][base.py:57 todd.IterBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:27:10,912 35174:140704541179520][log.py:93 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([4, 1]), 'y': tensor([8, 2])} weight=0.000 loss=5.000
[2024-02-23 18:27:10,913 35174:140704541179520][checkpoint.py:80 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp5g2sg2lp/checkpoint_callback/checkpoints/iter_1
[2024-02-23 18:27:10,917 35174:140704541179520][log.py:93 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/8] batch={'x': tensor([6, 7]), 'y': tensor([12, 14])} weight=0.012 loss=12.919
[2024-02-23 18:27:10,918 35174:140704541179520][checkpoint.py:80 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp8


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp5g2sg2lp
└── checkpoint_callback
    └── checkpoints
        ├── iter_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_4
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_5
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_6
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── opt

[2024-02-23 18:27:11,413 35174:140704541179520][checkpoint.py:54 todd.IterBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp5g2sg2lp/checkpoint_callback/checkpoints/iter_5
[2024-02-23 18:27:11,420 35174:140704541179520][base.py:65 todd.IterBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-23 18:27:11,421 35174:140704541179520][base.py:57 todd.IterBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:27:11,424 35174:140704541179520][log.py:93 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [6/8] batch={'x': tensor([3, 7]), 'y': tensor([ 6, 14])} weight=0.138 loss=9.312
[2024-02-23 18:27:11,425 35174:140704541179520][checkpoint.py:80 todd.IterBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp5g2sg2lp/checkpoint_callback/checkpoints/i

strategy.pth:
{}

optim.pth:
{'param_groups': [{'dampening': 0,
                   'differentiable': False,
                   'foreach': None,
                   'lr': 0.005,
                   'maximize': False,
                   'momentum': 0,
                   'nesterov': False,
                   'params': [0],
                   'weight_decay': 0}],
 'state': {0: {'momentum_buffer': None}}}

meta.pth:
{'iter_': 5}

model.pth:
OrderedDict([('_weight', tensor(0.1375))])

callbacks.pth:
{'callbacks': [{}, {}, {}]}



In [22]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="checkpoint_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=2),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_8 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_8'
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_8),
        )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    iter_10 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_10'
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_10),
        )
    runner.run()


[2024-02-23 18:27:11,475 35174:140704541179520][base.py:57 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:27:11,476 35174:140704541179520][log.py:99 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [1/3]
[2024-02-23 18:27:11,478 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/15] batch={'x': tensor([10,  1]), 'y': tensor([20,  2])} weight=0.000 loss=11.000
[2024-02-23 18:27:11,480 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/15] batch={'x': tensor([5, 3]), 'y': tensor([10,  6])} weight=0.027 loss=7.890
[2024-02-23 18:27:11,481 35174:140704541179520][checkpoint.py:80 todd.EpochBasedTrainer.checkpoint_callback _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp46q6bkgj/checkpoint_callback/checkpoints/iter_2
[2024-02-23 18:27:11,484 35174:140


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp46q6bkgj
└── checkpoint_callback
    └── checkpoints
        ├── iter_10
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_12
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_14
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_4
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_6
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── 

[2024-02-23 18:27:11,956 35174:140704541179520][checkpoint.py:54 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp46q6bkgj/checkpoint_callback/checkpoints/iter_8
[2024-02-23 18:27:11,959 35174:140704541179520][base.py:65 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-23 18:27:11,960 35174:140704541179520][base.py:57 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:27:11,961 35174:140704541179520][log.py:99 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [2/3]
[2024-02-23 18:27:11,964 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [9/15] batch={'x': tensor([6, 4]), 'y': tensor([12,  8])} weight=0.218 loss=8.913
[2024-02-23 18:27:11,966 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.checkpoint_c


--------------------



[2024-02-23 18:27:12,442 35174:140704541179520][checkpoint.py:54 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp46q6bkgj/checkpoint_callback/checkpoints/iter_10
[2024-02-23 18:27:12,445 35174:140704541179520][base.py:65 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-23 18:27:12,446 35174:140704541179520][base.py:57 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:27:12,447 35174:140704541179520][log.py:99 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [3/3]
[2024-02-23 18:27:12,450 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [11/15] batch={'x': tensor([6, 2]), 'y': tensor([12,  4])} weight=0.270 loss=6.920
[2024-02-23 18:27:12,452 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.checkpoint

In [23]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="checkpoint_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1, by_epoch=True),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    epoch_2 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'epoch_2'
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(epoch_2),
        )
    runner.run()


[2024-02-23 18:27:12,498 35174:140704541179520][base.py:57 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:27:12,499 35174:140704541179520][log.py:99 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [1/3]
[2024-02-23 18:27:12,501 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/15] batch={'x': tensor([3, 1]), 'y': tensor([6, 2])} weight=0.000 loss=4.000
[2024-02-23 18:27:12,503 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [2/15] batch={'x': tensor([8, 5]), 'y': tensor([16, 10])} weight=0.010 loss=12.935
[2024-02-23 18:27:12,505 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [3/15] batch={'x': tensor([2, 7]), 'y': tensor([ 4, 14])} weight=0.042 loss=8.809
[2024-02-23 18:27:12,507 35174:140704541179520][log.py:93 todd.EpochBa


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp8jm3x0vu
└── checkpoint_callback
    └── checkpoints
        ├── epoch_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest -> epoch_3

7 directories, 15 files



[2024-02-23 18:27:12,968 35174:140704541179520][checkpoint.py:54 todd.EpochBasedTrainer.checkpoint_callback init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp8jm3x0vu/checkpoint_callback/checkpoints/epoch_2
[2024-02-23 18:27:12,974 35174:140704541179520][base.py:65 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-23 18:27:12,976 35174:140704541179520][base.py:57 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:27:12,978 35174:140704541179520][log.py:99 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [3/3]
[2024-02-23 18:27:12,982 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [11/15] batch={'x': tensor([1, 8]), 'y': tensor([ 2, 16])} weight=0.275 loss=7.763
[2024-02-23 18:27:12,984 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.checkpoint

### Monitor

In [24]:
class CustomError(RuntimeError):
    pass

In [25]:
@todd.RunnerRegistry.register_()
class FaultyValidator(todd.runners.Validator):

    def _run_iter(self, *args, **kwargs) -> NoReturn:
        raise CustomError("faulty runner")

In [26]:
config = todd.Config(
    type='FaultyValidator',
    name='monitor_callback',
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(type='MonitorCallback'),
        dict(type='LogCallback', interval=5, with_file_handler=True),
    ],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    try:
        runner.run()
    except CustomError as e:
        pass

    !echo
    !cat {work_dirs}/monitor_callback/*.log


[2024-02-23 18:27:13,032 35174:140704541179520][base.py:57 todd.FaultyValidator.monitor_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:27:13,035 35174:140704541179520][monitor.py:26 todd.FaultyValidator.monitor_callback __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x14c1dc050>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 255, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_35174/1715875531.py", line 5, in _run_iter
    raise CustomError("faulty runner")
CustomError: faulty runner



[2024-02-23 18:27:13,032 35174:140704541179520][base.py:57 todd.FaultyValidator.monitor_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:27:13,035 35174:140704541179520][monitor.py:26 todd.FaultyValidator.monitor_callback __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x14c1dc050>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 255, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_35174/1715875531.py", line 5, in _run_iter
    raise CustomError("faulty runner")
CustomError: faulty runner


### Priorities

## Strategies

In [27]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="strategy_load_model_from",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1, by_epoch=True),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    epoch_2 = (pathlib.Path(work_dirs) / 'strategy_load_model_from' / 'checkpoints' / 'epoch_2' / 'model.pth')
    runner: todd.runners.BaseRunner = \
        todd.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.strategy.load_model_from(epoch_2)
    runner.run()


[2024-02-23 18:27:13,341 35174:140704541179520][base.py:57 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:27:13,342 35174:140704541179520][log.py:99 todd.EpochBasedTrainer.strategy_load_model_from before_run_epoch] INFO: Epoch [1/3]
[2024-02-23 18:27:13,346 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [1/15] batch={'x': tensor([1, 3]), 'y': tensor([2, 6])} weight=0.000 loss=4.000
[2024-02-23 18:27:13,347 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [2/15] batch={'x': tensor([6, 2]), 'y': tensor([12,  4])} weight=0.010 loss=7.960
[2024-02-23 18:27:13,349 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [3/15] batch={'x': tensor([10,  8]), 'y': tensor([20, 16])} weight=0.030 loss=17.730
[2024-02-23 18:27:13,351 35174:140704541179


--------------------



[2024-02-23 18:27:13,865 35174:140704541179520][base.py:57 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-02-23 18:27:13,866 35174:140704541179520][base.py:80 todd.EpochBasedTrainer.strategy_load_model_from load_model_from] INFO: Loading model from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp91d3imrz/strategy_load_model_from/checkpoints/epoch_2/model.pth
[2024-02-23 18:27:13,868 35174:140704541179520][base.py:65 todd.EpochBasedTrainer.strategy_load_model_from load_model_state_dict] INFO: <All keys matched successfully>
[2024-02-23 18:27:13,869 35174:140704541179520][log.py:99 todd.EpochBasedTrainer.strategy_load_model_from before_run_epoch] INFO: Epoch [1/3]
[2024-02-23 18:27:13,873 35174:140704541179520][log.py:93 todd.EpochBasedTrainer.strategy_load_model_from after_run_iter] INFO: Iter [1/15] batch={'x': tensor([7, 2]), 'y': tensor([14,  4])} weight=0.275 loss=7.762
[2024-02-23 18:27:13,875 35174:1407045411

## Dry Run

In [28]:
todd.Store.DRY_RUN = True